In [ ]:
import pandas as pd
from sqlalchemy.dialects.mssql import pymssql
import pymssql
import threading
import time, datetime
from config import cloudM,cloudMpassword,sqluser,sqlpass,servername

conn = pymssql.connect(server=servername, user=sqluser, password=sqlpass, database='CoronaVirus') 
cursor = conn.cursor()
qry='SELECT * from cvirus' #4.28M

In [ ]:
start=time.time()
df_minrating=pd.read_sql(qry,conn)
df_minrating.to_csv('minrating.csv',index=False)
end=time.time()
print('Total time is ' , end - start)

In [ ]:
#chunk_range=((1,1000000),(1000001,2000000),(2000001,3000000),(3000001,4000000),(4000001,4285519))
chunk_range=((1,200000),(200001,400000),(400001,600000),(600001,800000),(800001,1000000),(1000001,1200000),(1200001,1400000),(1400001,1600000),(1600001,1800000),(1800001,2000000),(2000001,2200000),(2200001,2400000))
newsql = 'select * from cvirus with (nolock) where id between %(start)s and %(end)s'
def create_csv(chunk):
    print('Starting execution')
    conn = pymssql.connect(server=servername, user=sqluser, password=sqlpass, database='CoronaVirus') 
    df_minrating=pd.read_sql(newsql,conn,params={'start':chunk[0],'end':chunk[1]})
    df_minrating.to_csv('chunk_' + str(chunk[0]) + '.csv',index=False)

In [ ]:
thread_list=[]
start=time.time()
for i in range(len(chunk_range)):
    t=threading.Thread(target=create_csv,args=(chunk_range[i],))
    thread_list.append(t)
for thread in thread_list:
    thread.start()
for thread in thread_list:
    thread.join()
end=time.time()
print('Total time is ' , end - start)

In [7]:
import multiprocessing
from multiprocessing import Process
def create_csv(chunk=(1,200000)):
    print('Starting execution')
    conn = pymssql.connect(server=servername, user=sqluser, password=sqlpass, database='CoronaVirus') 
    df_minrating=pd.read_sql(newsql,conn,params={'start':chunk[0],'end':chunk[1]})
    df_minrating.to_csv('chunk_' + str(chunk[0]) + '.csv',index=False)
    
if __name__ == "__main__":  # confirms that the code is under main function
    chunk_range1=[(1,200000),(200001,400000),(400001,600000),(600001,800000),(800001,1000000),(1000001,1200000),(1200001,1400000),(1400001,1600000),(1600001,1800000),(1800001,2000000),(2000001,2200000),(2200001,2400000)]
    names = chunk_range1
    procs = []
    #proc = Process(target=print_func)  # instantiating without any argument
    proc = Process(target=create_csv)
    procs.append(proc)
    proc.start()

    # instantiating process with arguments
    #for name in names:
    for i in range(len(chunk_range1)):
        # print(name)
        proc = Process(target=create_csv, args=((chunk_range1[i],)))
        procs.append(proc)
        proc.start()

    # complete the processes
    for proc in procs:
        proc.join()

In [10]:
from multiprocessing import Process


def print_func(continent='Asia'):
    return print('The name of continent is : ', continent)

if __name__ == "__main__":  # confirms that the code is under main function
    num_processor=4
    pool = multiprocessing.Pool(num_processor)
    names = ['America', 'Europe', 'Africa']
    procs = []
    proc = Process(target=print_func)  # instantiating without any argument
    procs.append(proc)
    proc.start()

    # instantiating process with arguments
    for name in names:
        # print(name)
        proc = Process(target=print_func, args=(name,))
        procs.append(proc)
        proc.start()

    # complete the processes
    for proc in procs:
        proc.join()

In [6]:
from multiprocessing import Lock, Process, Queue, current_process
import time
import queue # imported for using queue.Empty exception





def main():
    def do_job(tasks_to_accomplish, tasks_that_are_done):
        while True:
            try:
                '''
                    try to get task from the queue. get_nowait() function will 
                    raise queue.Empty exception if the queue is empty. 
                    queue(False) function would do the same task also.
                '''
                task = tasks_to_accomplish.get_nowait()
            except queue.Empty:

                break
            else:
                '''
                    if no exception has been raised, add the task completion 
                    message to task_that_are_done queue
                '''
                print(task)
                tasks_that_are_done.put(task + ' is done by ' + current_process().name)
                time.sleep(.5)
    return True
    number_of_task = 10
    number_of_processes = 4
    tasks_to_accomplish = Queue()
    tasks_that_are_done = Queue()
    processes = []

    for i in range(number_of_task):
        tasks_to_accomplish.put("Task no " + str(i))

    # creating processes
    for w in range(number_of_processes):
        p = Process(target=do_job, args=(tasks_to_accomplish, tasks_that_are_done))
        processes.append(p)
        p.start()

    # completing process
    for p in processes:
        p.join()

    # print the output
    while not tasks_that_are_done.empty():
        print(tasks_that_are_done.get())

    return True


if __name__ == '__main__':
    main()